In [1]:
import os
#os.environ["OPENAI_API_KEY"] = ""
# os.environ["HF_TOKEN"] = 

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
import numpy as np

In [3]:
# 데이터셋 로드
klue_sts_train = load_dataset('klue', 'sts', split='train')
klue_sts_validation = load_dataset('klue', 'sts', split='validation')

README.md:   0%|          | 0.00/22.5k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.52M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/68.8k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/11668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/519 [00:00<?, ? examples/s]

In [4]:
# 필요 없는 열 제거
klue_sts_train_data = klue_sts_train.remove_columns(["guid", "source"])
klue_sts_validation_data = klue_sts_validation.remove_columns(["guid", "source"])

In [5]:
klue_sts_train[0]

{'guid': 'klue-sts-v1_train_00000',
 'source': 'airbnb-rtt',
 'sentence1': '숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.',
 'sentence2': '숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.',
 'labels': {'label': 3.7, 'real-label': 3.714285714285714, 'binary-label': 1}}

In [6]:
# train 데이터 크기 줄이기 (20%만 사용)
train_dataset = klue_sts_train_data.select(range(0, int(len(klue_sts_train_data) * 0.2)))


In [7]:
# 데이터 분할: 80% train, 20% valid
train_test_split = train_dataset.train_test_split(test_size=0.2, shuffle=True, seed=24)
train_dataset = train_test_split['train']
valid_dataset = train_test_split['test']

In [8]:
# 토크나이저 로드
model_id = 'klue/roberta-base'
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [9]:
# 토크나이저 로드
model_id = 'klue/roberta-base'
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [10]:
# 데이터 토큰화 함수
def tokenize_function(examples):
    return tokenizer(
        examples["sentence1"], 
        examples["sentence2"], 
        padding="max_length", 
        truncation=True, 
        max_length=128  # max_length를 지정하여 패딩할 길이 설정
    )
    encoding["labels"] = examples["score"]  # 'score'를 레이블로 사용
    return encoding



In [11]:
# 데이터셋에 토크나이저 적용
train_dataset = train_dataset.map(tokenize_function, batched=True)
valid_dataset = valid_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1866 [00:00<?, ? examples/s]

Map:   0%|          | 0/467 [00:00<?, ? examples/s]

In [12]:
# 첫 번째 샘플의 키와 값을 출력하여 데이터 구조 확인
print(train_dataset[0].keys())


dict_keys(['sentence1', 'sentence2', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'])


In [13]:
def convert_labels_to_binary(example):
    # labels가 딕셔너리 형식일 경우 'score' 값으로 변환
    if isinstance(example["labels"], dict):
        score = example["labels"].get("score", 0)
    else:
        score = example["labels"]
    
    # score 값을 이진 레이블로 변환
    example["labels"] = 1 if score > 0 else 0
    return example


train_dataset = train_dataset.map(convert_labels_to_binary)
valid_dataset = valid_dataset.map(convert_labels_to_binary)


# score 값을 기준으로 이진 레이블로 변환하는 함수
# score > 0이면 1, 그외는 0으로 레이블 설정
# 문장유사도가 0보다 크면 1, 그렇지 않으면 0으로 지성// 이진분류 분제 변환

Map:   0%|          | 0/1866 [00:00<?, ? examples/s]

Map:   0%|          | 0/467 [00:00<?, ? examples/s]

In [14]:
# 훈련 설정
training_args = TrainingArguments(
    output_dir='roberta-base-klue-sts',
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy='epoch',
    learning_rate=0.001,
    push_to_hub=False,
    logging_steps=1,
    report_to="none",
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [15]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # 모델의 예측 값이 확률 분포일 경우, 가장 큰 값을 선택하여 이진 레이블로 변환
    predictions = logits.argmax(axis=-1)  # 다중 클래스일 경우 확률값을 가장 큰 클래스로 변환
    return {"pearson": np.corrcoef(predictions, labels)[0, 1]}

#pearson 상관계수 계산하여 모델 성능 평가

In [16]:
# 모델 로드
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=2)

config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
# Trainer 객체 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

<ipython-input-17-1ea75a54aff0>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [18]:
print(train_dataset[0])  # 첫 번째 샘플 확인


{'sentence1': '어떤 중국인은 100유로 냈다는 사람도 있었답니다.', 'sentence2': '방은 1개에 여자 두사람이 잘 수 있는 침대 1개 그리고 매트리스 1개도 있었어요 ,', 'labels': 0, 'input_ids': [0, 3711, 8116, 2073, 3911, 2298, 2200, 738, 4000, 3611, 2119, 1513, 2359, 13160, 18, 2, 1129, 2073, 21, 2019, 2170, 3883, 864, 11336, 2052, 1521, 1295, 1513, 2259, 6849, 21, 2019, 3673, 20852, 21, 2019, 2119, 1513, 2359, 10283, 16, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [19]:
# 파인튜닝 시작
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Pearson
1,0.000000,0.000000,nan


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]


TrainOutput(global_step=117, training_loss=0.005611410140151845, metrics={'train_runtime': 41.5595, 'train_samples_per_second': 44.899, 'train_steps_per_second': 2.815, 'total_flos': 122741307325440.0, 'train_loss': 0.005611410140151845, 'epoch': 1.0})

In [20]:
# 평가
trainer.evaluate(valid_dataset)

{'eval_loss': 0.0,
 'eval_pearson': nan,
 'eval_runtime': 2.6752,
 'eval_samples_per_second': 174.567,
 'eval_steps_per_second': 11.214,
 'epoch': 1.0}

In [21]:
from transformers import pipeline

repo_id = 'jwyzfe/roberta-base-sts-classification'
model_pipline = pipeline('text-classification', model=model_id)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


In [22]:
model_pipline(train_dataset[4:10]['sentence1'])


[{'label': 'LABEL_1', 'score': 0.5533674359321594},
 {'label': 'LABEL_1', 'score': 0.5491771101951599},
 {'label': 'LABEL_1', 'score': 0.5510575175285339},
 {'label': 'LABEL_1', 'score': 0.5658992528915405},
 {'label': 'LABEL_1', 'score': 0.5568838715553284},
 {'label': 'LABEL_1', 'score': 0.5488966107368469}]